In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv("../原始数据/cleaned_senti_reviews.csv")

In [4]:
df.head(2)

,Unnamed: 0,reviewer_id,listing_id,date,reviewer_name,comments,name,description,neighborhood_overview,host_id,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,reviews_per_month,polarity_score,sentiment
0,0,8557223,49091,2013-10-21,Jared,Fran was absolutely gracious and welcoming. Ma...,COZICOMFORT LONG TERM STAY ROOM 2,This is Room No. 2.(available after Jan 15 201...,NaN,266763,...,10.0,10.0,10.0,8.0,8.0,f,2,0.02,0.9615,pos
1,1,1356099,50646,2014-04-18,James,A comfortable room in a smart condo developmen...,Pleasant Room along Bukit Timah,Fully furnished bedroom with a nice view on th...,The serenity & quiet surrounding makes it an i...,227796,...,10.0,10.0,10.0,9.0,9.0,f,1,0.30,0.9484,pos


### 1. 筛选特征 

#### 1.1 主观筛选 

In [5]:
delete_columns=['reviewer_name','name','neighborhood_overview','host_id','host_about','host_neighbourhood',
                'host_has_profile_pic', 'host_identity_verified','neighbourhood_cleansed', 'neighbourhood_group_cleansed',
                'calendar_updated','number_of_reviews'
               ]

In [6]:
df2=df.drop(delete_columns,axis=1)

#### 1.2 客观筛选：测试和sentimental score的correlation   <font color=#008000>未完成</font>

### 2. 处理特征 

#### date特征转化为date_time类型

In [7]:
df2["date"]=pd.to_datetime(df2.date)

#### 删除comments空值

In [8]:
df2.dropna(subset=["comments"],inplace=True)

#### price特征：字符串转化为小数  表示住一晚上的价格

In [9]:
df2["price"]=df2.price.apply(lambda x:float(x[1:].replace(",","")))

In [10]:
df2.price.value_counts()

60.0      4112
50.0      3543
85.0      3445
70.0      2775
120.0     2632
          ... 
396.0        1
433.0        1
2943.0       1
530.0        1
597.0        1
Name: price, Length: 545, dtype: int64

#### review_scores_rating系列：删除空值  并 整合这些特征feature cross <font color=#008000>(未完成)</font>

In [11]:
df2[['review_scores_rating',
       #'review_scores_accuracy', 'review_scores_cleanliness',
       #'review_scores_checkin', 'review_scores_communication',
       #'review_scores_location', 'review_scores_value'
    ]].isnull().value_counts()

review_scores_rating
False                   130628
True                         4
dtype: int64

我们发现缺失值较少，因此这些特征的空值的样本都删除

In [12]:
df2.dropna(subset=['review_scores_rating','review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value'],inplace=True)

In [13]:
df2.shape

(130602, 46)

### 3. 提取最终特征

In [14]:
feature_selected=['reviewer_id', 'listing_id', 'date', 'comments', 'price',
                  'review_scores_rating','review_scores_accuracy', 'review_scores_cleanliness',
                  'review_scores_checkin', 'review_scores_communication','review_scores_location',
                  'review_scores_value']

In [15]:
customers=df2[feature_selected]

In [16]:
customers.head(2)

,reviewer_id,listing_id,date,comments,price,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,8557223,49091,2013-10-21,Fran was absolutely gracious and welcoming. Ma...,81.0,94.0,10.0,10.0,10.0,10.0,8.0,8.0
1,1356099,50646,2014-04-18,A comfortable room in a smart condo developmen...,80.0,91.0,9.0,10.0,10.0,10.0,9.0,9.0


In [17]:
customers.shape

(130602, 12)

### 3.导出文件

In [18]:
customers.to_csv("../终版数据/customers_final_version.csv",index=False)